In [1]:
import pandas as pd

In [2]:
import numpy as np

In [7]:
trfname=input("Write your train data file name : ")
#tstfname=input("Write your test data file name : ")
sep=input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")
if sep=='1':
    form=' '
elif sep=='2':
    form=','


hd=input("Select the data header format(1 = with header or 2 = no header): ")
if hd=='1':
    header='infer'
elif hd=='2':
    header=None

data= pd.read_csv(trfname, header = header , sep = form)

cl=input("Select column# where your y vlaues exist: ")
yCol = int(cl)-1
coff=input("Select your cutoff value: ")    
cutoff = float(coff)
misVal = input("input shape of your missing data")

Xraw = data.drop(yCol,1)
Y = data.iloc[:, yCol]

#test Y have 2 category
#if len(set(Y))!=2:
#    return('you must input data with binary y value')
# let max(Y) = 1 (True) , min(Y) = 0 (False)

#treat the missing value
X = Xraw.replace('?',np.nan).apply(pd.to_numeric)
n = X.shape[0]

# count the number of unique values, when it is less or equal than 10 treat the variable as categorical
unqCnt = []
for i in range(X.shape[1]):
    unqCnt += [len(set(X.iloc[:,i]))]
unqCnt = np.array(unqCnt)

cateVarInd = np.where(uqcnt<=10)[0] # index of the categoral values
numerVarInd = np.where(uqcnt>10)[0] # index of the numerical values

# for gaussian naive bayes we need to calculate the variance and mean of numerical variables
X1 = X[list(map(bool, Y - min(Y)))] # Y = max(Y)
X0 = X[list(map(bool, Y - max(Y)))] # Y = min(Y)
#True
mu1 = dict(np.mean(X1,axis = 0)[numerVarInd])
std1 = dict(np.std(X1,axis = 0)[numerVarInd])
#False
mu0 = dict(np.mean(X0,axis = 0)[numerVarInd])
std0 = dict(np.std(X0,axis = 0)[numerVarInd])


# when true X ~ N (mu1, var1)
# P( X =x | True) =  (1/(sqrt(2pie)*std))*exp((-(x-mu)^2)/(2*std^2))

# make probability table
cst= dict()
for i in cateVarInd:
    cst.update({i:pd.crosstab(X.iloc[:,i],Y)})
    for j in range(2):
        cst[i].iloc[:,j] = cst[i].iloc[:,j]/sum(cst[i].iloc[:,j])
        
# calculate prob
def calcDPT(row):
    p = 1
    for j in cateVarInd:
        if np.isnan(row[j]):
            p = p
        else:
            p = p * cst[j].loc[row[j]][max(Y)]
    return(p)

def calcDPF(row):
    p = 1
    for j in cateVarInd:
        if np.isnan(row[j]):
            p = p
        else:
            p = p * cst[j].loc[row[j]][min(Y)]
    return(p)

def calcNPT(row):
    p = 1
    for j in numerVarInd:
        if np.isnan(row[j]):
            p = p
        else:
            p = p * (1/(((2*np.pi)**0.5)*std1[j]))*np.exp((-(row[j]-mu1[j])**2)/(2*std1[j]**2))
    return(p)

def calcNPF(row):
    p = 1
    for j in numerVarInd:
        if np.isnan(row[j]):
            p = p
        else:
            p = p * (1/(((2*np.pi)**0.5)*std0[j]))*np.exp((-(row[j]-mu0[j])**2)/(2*std0[j]**2))
    return(p)

def calcT(row):
    return(calcDPT(row)*calcNPT(row))

def calcF(row):
    return(calcDPF(row)*calcNPF(row))

def getTprob(row):
    return calcT(row)/(calcT(row)+calcF(row))

predProb = []
for i in range(X.shape[0]):
    predProb.append(getTprob(X.iloc[i,:]))
    
def gety(x):
    if x>=cutoff:
        return max(Y)
    elif x<cutoff:
        return min(Y)
    
Yhat = np.array(list(map(gety,predProb)))
yprob = list(map(lambda x : round(x,3),predProb))

cont = pd.crosstab(Y,Yhat)
acc = sum(np.diag(cont))/n
spe = cont.iloc[0,0]/sum(cont.iloc[0,:])
sens = cont.iloc[1,1]/sum(cont.iloc[1,:])
predy = pd.DataFrame([range(1,(len(X)+1)),Y,Yhat,yprob]).transpose()
predy.columns=['ID','Actual class','Resub pred','Pred Prob']
cont.columns.name = 'Resub Class'
cont.index.name = 'Actual Class'

#console output
print(predy.to_string(index=False))
print(cont)
print("Overall accuracy = %0.3f" %(acc))
print("Sensitivity = %0.3f" %(sens))
print("Specificity = %0.3f" %(spe))

#file output
fname = input('Write your output file name : ')
out = open(fname, 'w')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(predy.to_string(index=False),file=out)
    print("",file=out)
    print("Confusion Matrix (Resubstitution)",file=out)
    print(cont,file=out)
    print("",file=out)
    print("Model Summary (Resubstitution)",file=out)
    print("Overall accuracy = %0.3f" %(acc),file=out)
    print("Sensitivity = %0.3f" %(sens),file=out)
    print("Specificity = %0.3f" %(spe),file=out)
out.close()


Write your train data file name : heart.dat
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): 1
Select the data header format(1 = with header or 2 = no header): 2
Select column# where your y vlaues exist: 14
Select your cutoff value: 0.5
input shape of your missing data?


'\n#test Y have 2 category\n#if len(set(Y))!=2:\n#    return(\'you must input data with binary y value\')\n# let max(Y) = 1 (True) , min(Y) = 0 (False)\n\n#treat the missing value\nX = Xraw.replace(\'?\',np.nan).apply(pd.to_numeric)\nn = X.shape[0]\n\n# count the number of unique values, when it is less or equal than 10 treat the variable as categorical\nunqCnt = []\nfor i in range(X.shape[1]):\n    unqCnt += [len(set(X.iloc[:,i]))]\nunqCnt = np.array(unqCnt)\n\ncateVarInd = np.where(uqcnt<=10)[0] # index of the categoral values\nnumerVarInd = np.where(uqcnt>10)[0] # index of the numerical values\n\n# for gaussian naive bayes we need to calculate the variance and mean of numerical variables\nX1 = X[list(map(bool, Y - min(Y)))] # Y = max(Y)\nX0 = X[list(map(bool, Y - max(Y)))] # Y = min(Y)\n#True\nmu1 = dict(np.mean(X1,axis = 0)[numerVarInd])\nstd1 = dict(np.std(X1,axis = 0)[numerVarInd])\n#False\nmu0 = dict(np.mean(X0,axis = 0)[numerVarInd])\nstd0 = dict(np.std(X0,axis = 0)[numerVarInd

In [8]:
Xraw

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,35,1,4,120,198,0,1,130,1,1.6,2,1,3
1,43,1,4,120,177,0,3,120,1,2.5,2,1,3
2,62,0,4,160,164,0,3,145,0,6.2,3,4,3
3,67,1,3,152,212,0,3,150,0,0.8,2,1,3
4,44,1,4,110,197,0,3,177,0,0.0,1,2,1
5,58,0,4,170,225,1,3,146,1,2.8,2,3,2
6,63,0,4,108,269,0,1,169,1,1.8,2,3,1
7,61,0,4,145,307,0,3,146,1,1.0,2,1,3
8,54,1,4,124,266,0,3,109,1,2.2,2,2,3
9,54,1,4,120,188,0,1,113,0,1.4,2,2,3


In [9]:
std1

NameError: name 'std1' is not defined